<a href="https://colab.research.google.com/github/sangjinsu/recommendation-system/blob/main/content_based_recommendation_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Read Data

In [33]:
movies = pd.read_csv('/content/drive/MyDrive/추천 시스템 입문반/data/ml-latest-small/movies.csv')
ratings = pd.read_pickle('/content/drive/MyDrive/추천 시스템 입문반/data/ml-latest-small/ratings_updated.p')
genres = pd.read_pickle('/content/drive/MyDrive/추천 시스템 입문반/data/ml-latest-small/genres.p')

In [34]:
ratings.tail()

,userId,movieId,rating,timestamp
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


In [35]:
genres.sample()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
58156,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Processing

In [36]:
ratings = ratings.merge(genres, left_on='movieId', right_index=True)
ratings.sample()

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
52411,341,435,3.0,2015-08-16 18:55:09,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [37]:
ratings = ratings.replace(0, np.nan)
ratings

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,2000-07-30 18:45:03,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,5,1,4.0,1996-11-08 06:36:02,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
874,7,1,4.5,2005-01-25 06:52:26,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1434,15,1,2.5,2017-11-13 12:59:30,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1667,17,1,4.5,2011-05-18 05:28:03,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100820,610,160341,2.5,2016-11-19 08:55:49,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
100821,610,160527,4.5,2016-11-19 08:43:18,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100823,610,160836,3.0,2017-05-03 20:53:14,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
100827,610,163937,3.5,2017-05-03 21:59:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


### Train test Split

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
train, test = train_test_split(ratings, random_state=42, test_size=.1)

In [40]:
print(train.shape)
print(test.shape)

(90752, 24)
(10084, 24)


## Item Profile

In [41]:
genres

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193583,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193585,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### User Profile

In [42]:
genres_cols = genres.columns
genres_cols

Index(['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [43]:
for cols in genres_cols:
  train[cols] = train[cols] * train['rating']

In [44]:
train

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
51127,330,2959,4.0,2010-10-01 03:36:47,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
21133,140,376,3.0,2000-08-23 16:45:18,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
100240,610,52328,4.0,2016-11-19 07:54:45,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN
96114,603,52,1.0,2000-07-09 21:30:36,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
64701,414,59141,4.5,2010-05-16 02:34:46,NaN,NaN,NaN,NaN,4.5,4.5,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36445,249,1089,4.0,2012-09-04 11:22:25,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN
12263,75,44199,3.0,2006-09-22 23:35:09,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
20096,132,3186,4.5,2006-09-11 17:54:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1671,17,110,4.5,2011-05-18 05:27:50,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN


In [45]:
train.groupby('userId')['Action'].mean()

userId
1      4.294872
2      4.142857
3      3.571429
4      3.272727
5      3.250000
         ...   
606    3.175373
607    3.724138
608    3.328629
609    3.111111
610    3.608742
Name: Action, Length: 610, dtype: float64

In [48]:
user_profile = train.groupby('userId')[genres_cols].mean()
user_profile

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,
1,NaN,4.294872,4.363636,4.689655,4.550000,4.293333,4.333333,NaN,4.593220,4.285714,NaN,3.250000,NaN,4.736842,4.214286,4.300000,4.228571,4.133333,4.500000,4.200000
2,NaN,4.142857,4.166667,NaN,NaN,4.000000,3.583333,4.333333,3.807692,NaN,NaN,NaN,3.750000,NaN,4.000000,4.500000,3.875000,3.583333,4.500000,3.500000
3,NaN,3.571429,2.950000,0.500000,0.500000,1.062500,0.500000,NaN,0.500000,4.333333,NaN,4.714286,NaN,0.500000,5.000000,0.500000,4.178571,4.083333,0.500000,NaN
4,NaN,3.272727,3.692308,4.000000,3.800000,3.515152,3.760000,4.000000,3.443478,3.684211,4.000000,4.250000,3.000000,4.000000,3.500000,3.379310,2.818182,3.485714,3.571429,3.875000
5,NaN,3.250000,3.333333,4.400000,4.125000,3.583333,3.818182,NaN,3.826087,4.333333,NaN,3.000000,3.666667,4.500000,NaN,3.250000,2.500000,3.714286,3.333333,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,3.175373,3.484252,3.671053,3.422222,3.553385,3.653153,3.800000,3.785489,3.619318,3.833333,3.397959,3.142857,3.712500,3.771605,3.721338,3.534247,3.522472,3.791667,3.468750
607,NaN,3.724138,3.500000,3.000000,3.352941,3.291667,3.850000,NaN,4.000000,3.555556,NaN,4.107143,5.000000,3.250000,4.571429,3.576923,3.218750,4.153846,4.500000,4.000000
608,NaN,3.328629,3.203704,3.021739,2.348101,2.713836,3.607692,3.000000,3.416000,2.950495,3.333333,3.280220,4.000000,2.655172,3.535714,2.842697,3.297297,3.545259,3.676471,2.636364


### My user Profile

In [52]:
user_profile

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,
1,NaN,4.294872,4.363636,4.689655,4.550000,4.293333,4.333333,NaN,4.593220,4.285714,NaN,3.250000,NaN,4.736842,4.214286,4.300000,4.228571,4.133333,4.500000,4.200000
2,NaN,4.142857,4.166667,NaN,NaN,4.000000,3.583333,4.333333,3.807692,NaN,NaN,NaN,3.750000,NaN,4.000000,4.500000,3.875000,3.583333,4.500000,3.500000
3,NaN,3.571429,2.950000,0.500000,0.500000,1.062500,0.500000,NaN,0.500000,4.333333,NaN,4.714286,NaN,0.500000,5.000000,0.500000,4.178571,4.083333,0.500000,NaN
4,NaN,3.272727,3.692308,4.000000,3.800000,3.515152,3.760000,4.000000,3.443478,3.684211,4.000000,4.250000,3.000000,4.000000,3.500000,3.379310,2.818182,3.485714,3.571429,3.875000
5,NaN,3.250000,3.333333,4.400000,4.125000,3.583333,3.818182,NaN,3.826087,4.333333,NaN,3.000000,3.666667,4.500000,NaN,3.250000,2.500000,3.714286,3.333333,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,3.175373,3.484252,3.671053,3.422222,3.553385,3.653153,3.800000,3.785489,3.619318,3.833333,3.397959,3.142857,3.712500,3.771605,3.721338,3.534247,3.522472,3.791667,3.468750
607,NaN,3.724138,3.500000,3.000000,3.352941,3.291667,3.850000,NaN,4.000000,3.555556,NaN,4.107143,5.000000,3.250000,4.571429,3.576923,3.218750,4.153846,4.500000,4.000000
608,NaN,3.328629,3.203704,3.021739,2.348101,2.713836,3.607692,3.000000,3.416000,2.950495,3.333333,3.280220,4.000000,2.655172,3.535714,2.842697,3.297297,3.545259,3.676471,2.636364


### Predict

In [59]:
sample = test.loc[84868]

In [60]:
sample_user = sample['userId']
sample_user_profile = user_profile.loc[sample_user]
sample_user_profile

(no genres listed)         NaN
Action                3.731343
Adventure             3.531250
Animation             3.833333
Children              4.300000
Comedy                3.631579
Crime                 3.895833
Documentary           4.000000
Drama                 3.638889
Fantasy               2.961538
Film-Noir             4.000000
Horror                4.250000
IMAX                  3.875000
Musical                    NaN
Mystery               3.944444
Romance               4.100000
Sci-Fi                3.640625
Thriller              3.782609
War                   4.187500
Western                    NaN
Name: 551, dtype: float64

In [67]:
movies[movies['movieId'] == 34162]

,movieId,title,genres
5938,34162,Wedding Crashers (2005),Comedy|Romance


In [64]:
print(sample['movieId'])
sample[genres_cols]

34162


(no genres listed)    NaN
Action                NaN
Adventure             NaN
Animation             NaN
Children              NaN
Comedy                  1
Crime                 NaN
Documentary           NaN
Drama                 NaN
Fantasy               NaN
Film-Noir             NaN
Horror                NaN
IMAX                  NaN
Musical               NaN
Mystery               NaN
Romance                 1
Sci-Fi                NaN
Thriller              NaN
War                   NaN
Western               NaN
Name: 84868, dtype: object

In [68]:
(sample_user_profile * sample[genres_cols]).mean()

3.8657894736842104

### 전체 데이터로 확장

In [73]:
from tqdm import tqdm_notebook

In [74]:
predict = []
for idx, row in tqdm_notebook(test.iterrows()):
  user = row['userId']
  predict.append((user_profile.loc[user] * row[genres_cols]).mean())

0it [00:00, ?it/s]

In [76]:
test['predict'] = predict

In [81]:
test.loc[test['predict'].isnull()] = train['rating'].mean()

In [82]:
test[test['predict'].isnull()]

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,predict


### Model validation

In [83]:
from sklearn.metrics import mean_squared_error

In [85]:
mse = mean_squared_error(test['rating'], test['predict'])
rmse = np.sqrt(mse)
rmse

0.9037778167769462